# Sepsis-3 evaluation in the MIMIC-III database

This notebook goes over the evaluation of the new Sepsis-3 guidelines in the MIMIC database. The goals of this analysis include:

1. Evaluating the Sepsis-3 guidelines in MIMIC using the same methodology as in the research paper
2. Evaluating the Sepsis-3 guidelines against ANGUS criteria
3. Assessing if there are interesting subgroup(s) which are missed by the criteria

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import sys
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import logit

from sepsis_utils import sepsis_utils as su
from sepsis_utils import roc_utils as ru

from sklearn.pipeline import Pipeline

# used for train/test splits
from sklearn.cross_validation import train_test_split

# used to impute mean for data
from sklearn.preprocessing import Imputer

# normalize the data
from sklearn import preprocessing

# logistic regression is our model of choice
from sklearn.linear_model import LogisticRegression

# used to create confusion matrix
from sklearn.metrics import confusion_matrix

from sklearn.cross_validation import cross_val_score

# used to calculate AUROC/accuracy
from sklearn import metrics

# for calibration curve of severity scores
from sklearn.calibration import calibration_curve

# default colours for prettier plots
col = [[0.9047, 0.1918, 0.1988],
    [0.2941, 0.5447, 0.7494],
    [0.3718, 0.7176, 0.3612],
    [1.0000, 0.5482, 0.1000],
    [0.4550, 0.4946, 0.4722],
    [0.6859, 0.4035, 0.2412],
    [0.9718, 0.5553, 0.7741],
    [0.5313, 0.3359, 0.6523]];
marker = ['v','o','d','^','s','o','+']
ls = ['-','-','-','-','-','s','--','--']
%matplotlib inline

from __future__ import print_function

In [ ]:
# load data
df = pd.read_csv('sepsis3-df.csv',sep=',')
df_mdl = pd.read_csv('sepsis3-design-matrix.csv',sep=',')

# define outcome
target_header = "hospital_expire_flag"
y = df[target_header].values == 1

# define the covariates to be added in the MFP model (used for table of AUROCs)
preds_header = ['sirs','sofa','lods','qsofa']

# Hospital mortality evaluation

In [ ]:
# define targets, angus critera
# define "predictions" according to the SEPSIS-3 guidelines:
#  suspicion of infection, qSOFA >= 2, and SOFA >= 2
yhat = (df.qsofa.values >= 2) & (df.sofa.values>=2)

print('\n SEPSIS-3 guidelines for hospital mortality \n')
# generate evaluation metrics
print('Accuracy = {}'.format(metrics.accuracy_score(y, yhat)))

su.print_cm(y, yhat) # print confusion matrix

We need to call an R script to run the fractional polynomial model on the data. All this is done in subfunctions, which evaluate:

* `print_auc_table` - the severity scores on their own
* `print_auc_table_baseline` - the severity scores in a vanilla regression
* `print_auc_table_given_preds` - the AUROC of the given predictions (which are generated using an MFP)

In [ ]:
# call a subprocess to run the R script
import subprocess
fn_in = "sepsis3-design-matrix.csv"
fn_out = "sepsis3-preds.csv"
rcmd = ["Rscript r-make-sepsis3-models.R", fn_in, fn_out, target_header]
err = subprocess.call(' '.join(rcmd), shell=True)
if err!=0:
    print('RScript returned error status {}.'.format(err))
else:
    # load in the predictions
    pred_baseline = pd.read_csv(fn_out, sep=',', header=0)
    
# loop through each severity score, build an MFP model for each

fn_in = "sepsis3-design-matrix.csv"
fn_out = "sepsis3-preds.csv"
preds_mfp = dict()
for p in preds_header:
    rcmd = ["Rscript r-make-sepsis3-models.R", fn_in, fn_out, target_header, p]
    err = subprocess.call(' '.join(rcmd), shell=True)
    if err!=0:
        print('RScript returned error status {}.'.format(err))
    else:
        # load in the predictions
        pred = pd.read_csv(fn_out, sep=',', header=0)
        preds_mfp[p] = pred.values[:,0]

In [ ]:
# reproduce the confusion matrix plot
su.print_auc_table(df_mdl, preds_header, target_header)
su.print_auc_table_baseline(df_mdl, preds_header, target_header)
su.print_auc_table_given_preds(preds_mfp, y, preds_header=preds_header) # optional argument fixes order of output

## ROC curves

In [ ]:
# ROC for qSOFA
fpr_qsofa, tpr_qsofa, thresholds_qsofa = metrics.roc_curve(y, df.qsofa.values)
auc_qsofa = metrics.auc(fpr_qsofa, tpr_qsofa)

# ROC for SOFA
fpr_sofa, tpr_sofa, thresholds_sofa = metrics.roc_curve(y, df.sofa.values)
auc_sofa = metrics.auc(fpr_sofa, tpr_sofa)


# ROC for SEPSIS-3
fpr_s3, tpr_s3, thresholds_s3 = metrics.roc_curve(y, (df.qsofa.values >= 2) & (df.sofa.values >= 2))
auc_s3 = metrics.auc(fpr_s3, tpr_s3)

# ROC for SIRS
fpr_sirs, tpr_sirs, thresholds_sirs = metrics.roc_curve(y, df.sirs.values)
auc_sirs = metrics.auc(fpr_sirs, tpr_sirs)

# plot the data
plt.figure(figsize=[9,9])
plt.plot(fpr_qsofa, tpr_qsofa, 'o:',
         color=col[0], linewidth=2, markersize=10,
         label='qSOFA (AUC = %0.2f)' % auc_qsofa)
plt.plot(fpr_sofa, tpr_sofa, '^-',
         color=col[1], linewidth=2, markersize=10,
         label='SOFA (AUC = %0.2f)' % auc_sofa)
plt.plot(fpr_sirs, tpr_sirs, 's-',
         color=col[2], linewidth=2, markersize=10,
         label='SIRS (AUC = %0.2f)' % auc_sirs)

# add in the combination of SIRS/SOFA
#plt.plot(fpr_s3, tpr_s3, 'd--',
#         color=col[3], linewidth=2, markersize=10,
#         label='SEPSIS-3 (AUC = %0.2f)' % auc_s3)

plt.legend(loc="lower right")

plt.plot([0,1], [0,1], '--',
         color=[0,0,0], linewidth=2)
# reformat the plot
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate',fontsize=14)
plt.ylabel('True Positive Rate',fontsize=14)
plt.title('ROC against ' + target_header,fontsize=14)
plt.show()

## Operating point statistics

In [ ]:
# define "predictions" according to the SEPSIS-3 guidelines:
#  suspicion of infection, qSOFA >= 2, and SOFA >= 2
yhat_all = [df.qsofa.values >= 2,
            df.sofa.values >= 2,
            df.sepsis3.values,
            df.sirs.values >= 2,
            df.lods.values >= 2]
yhat_names = ['qsofa','sofa','seps3','SIRS', 'LODS']

# define "targets", angus critera
y_all = [y, y, y, y, y]

stats_all = su.print_op_stats(yhat_all, y_all,
               yhat_names=yhat_names,
               header=target_header)

# Appendix

## Comparing binormal and empirical ROC


In [ ]:
y = df.hospital_expire_flag.values == 1
plt.figure(figsize=[9,9])

# === NORMAL EMPIRICALLY DERIVED ROC

# ROC for qSOFA
fpr_qsofa, tpr_qsofa, thresholds_qsofa = metrics.roc_curve(y, df.qsofa.values)
auc_qsofa = metrics.auc(fpr_qsofa, tpr_qsofa)

# ROC for SOFA
fpr_sofa, tpr_sofa, thresholds_sofa = metrics.roc_curve(y, df.sofa.values)
auc_sofa = metrics.auc(fpr_sofa, tpr_sofa)


# ROC for SEPSIS-3
fpr_s3, tpr_s3, thresholds_s3 = metrics.roc_curve(y, (df.qsofa.values >= 2) & (df.sofa.values >= 2))
auc_s3 = metrics.auc(fpr_s3, tpr_s3)

# ROC for SIRS
fpr_sirs, tpr_sirs, thresholds_sirs = metrics.roc_curve(y, df.sirs.values)
auc_sirs = metrics.auc(fpr_sirs, tpr_sirs)

# plot the data
plt.plot(fpr_qsofa, tpr_qsofa, 'o-',
         color=col[0], linewidth=2, markersize=10)
plt.plot(fpr_sofa, tpr_sofa, '^-',
         color=col[1], linewidth=2, markersize=10)
plt.plot(fpr_sirs, tpr_sirs, 's-',
         color=col[2], linewidth=2, markersize=10)


# === BINORMAL ESTIMATED ROC

# ROC for qSOFA
fpr_qsofa, tpr_qsofa, thresholds_qsofa = ru.binormal_roc(df.qsofa.values[y], df.qsofa.values[~y])
auc_qsofa2 = ru.binormal_auroc(df.qsofa.values[y], df.qsofa.values[~y])

# ROC for SOFA
fpr_sofa, tpr_sofa, thresholds_sofa = ru.binormal_roc(df.sofa.values[y], df.sofa.values[~y])
auc_sofa2 = ru.binormal_auroc(df.sofa.values[y], df.sofa.values[~y])

# ROC for SIRS
fpr_sirs, tpr_sirs, thresholds_sirs = ru.binormal_roc(df.sirs.values[y], df.sirs.values[~y])
auc_sirs2 = ru.binormal_auroc(df.sirs.values[y], df.sirs.values[~y])

# plot the data
plt.plot(fpr_qsofa, tpr_qsofa, '--',
         color=col[0], linewidth=2, markersize=10,
         label='qSOFA (AUC = {:0.3f} vs {:0.3f} for binorm)'.format(auc_qsofa, auc_qsofa2))
plt.plot(fpr_sofa, tpr_sofa, '--',
         color=col[1], linewidth=2, markersize=10,
         label='SOFA (AUC = {:0.3f} vs {:0.3f} for binorm)'.format(auc_sofa, auc_sofa2))
plt.plot(fpr_sirs, tpr_sirs, '--',
         color=col[2], linewidth=2, markersize=10,
         label='SIRS (AUC = {:0.3f} vs {:0.3f} for binorm)'.format(auc_sirs, auc_sirs2))

plt.legend(loc="lower right")

plt.plot([0,1], [0,1], '--',
         color=[0,0,0], linewidth=2)
# reformat the plot
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate',fontsize=14)
plt.ylabel('True Positive Rate',fontsize=14)
plt.title('Binormal ROC against hospital mortality',fontsize=14)
plt.show()

## Histograms comparing qSOFA in septic/non-septic population

In [ ]:
# histogram of the qSOFA values in septic/non-septic population
y = df.hospital_expire_flag.values == 1

qsofa_alive = df.qsofa.values[~y]
qsofa_dead = df.qsofa.values[y]

xi = [-0.5,0.5,1.5,2.5,3.5]

# plot the data
plt.figure(figsize=[9,9])
plt.hist(qsofa_alive, bins=xi, normed=True, color=col[0], alpha=0.5,
         label='qSOFA - ' + target_header + '=0')
plt.hist(qsofa_dead, bins=xi, normed=True, color=col[1], alpha=0.5,
         label='qSOFA - ' + target_header + '=1')

plt.legend(loc="upper right")

# reformat the plot
plt.xlim([-0.5,4.5])
#plt.ylim([-0.05, 1.05])
plt.xlabel('qSOFA',fontsize=14)
plt.ylabel('Count',fontsize=14)
plt.show()

In [ ]:
# histogram of the qSOFA values in septic/non-septic population
y = df.hospital_expire_flag.values == 1

qsofa_alive = df.qsofa.values[~y]
qsofa_dead = df.qsofa.values[y]

xi = [-0.5,0.5,1.5,2.5,3.5]

prevalence = np.mean(y)

# plot the data
plt.figure(figsize=[9,9])
n0, bins0, patches0 = plt.hist(qsofa_alive, bins=xi, normed=False, color=col[0], alpha=0.5,
         label='qSOFA - survived')
n1, bins1, patches1 = plt.hist(qsofa_dead, bins=xi, normed=False, color=col[1], alpha=0.5,
         label='qSOFA - died')

plt.legend(loc="upper right")

# reformat the plot
plt.xlim([-0.5,4.5])
#plt.ylim([-0.05, 1.05])
plt.xlabel('qSOFA',fontsize=14)
plt.ylabel('Count',fontsize=14)
#plt.title('ROC against hospital mortality',fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=[9,9])

N = len(y)
plt.bar(bins0[0:-1], 100.0*n0/N, width=1, color=col[0], alpha=0.5,
         label='qSOFA - survived')
plt.bar(bins1[0:-1], 100.0*n1/N, width=1, color=col[1], alpha=0.5,
         label='qSOFA - died')
plt.legend()
plt.ylabel('Percent of patients')
plt.show()